# Working Area

In [9]:
import requests
from bs4 import BeautifulSoup
import re
import json
from concurrent.futures import ThreadPoolExecutor, wait
from datetime import datetime
from pymongo import MongoClient



In [10]:
class StopWatch:
    start = None
    
    @staticmethod
    def start():
        StopWatch.start = datetime.now()
        
    @staticmethod    
    def split(break_point):
        try:
            split = datetime.now()
            time_elapsed = str(split - StopWatch.start)
            StopWatch.start = split
            print("break point = " + break_point + "time elapsed = " + time_elapsed)
        except:
            print("ValueError: StopWatch.split parameter must be string")
            
            

In [11]:
def get_url_links(post_urls, page):  
    """
    Read through HTML content and returns a list of url_links
    of each post in the category "politics" at Wetalk.tw
    """
    URL = "https://www.wetalk.tw/forum.php?mod=forumdisplay&fid=2&page={}".format(page)
    resp = requests.get(URL)

    soup = BeautifulSoup(resp.text, "html5lib")
    tmp_urls = [a.get('href') for a in soup.select('tbody > tr > th > a.xst')]
    post_urls.extend(tmp_urls[2:])
    return post_urls

In [12]:
def save_posts_info(post_urls):  
    """
    Save posts infomation to dictionary in the list
    """
    news_list = []
    count = 0

    for url in post_urls:
        news = {}
        resp_c = requests.get(url)
        soup_c = BeautifulSoup(resp_c.text, 'html5lib')   

        news["title"]    = soup_c.select("h1.ts")[0].text.replace('\n','')
        news["datetime"] = soup_c.select('span.date-show-info')[0].text.replace('發表','').strip()
        news["reporter"] = soup_c.select('a[itemprop="author"]')[0].text
        news["media"]    = "wetalk"
        news["category"] = "Forum"
        news["hash"]     = hash(resp_c.text)
        news["url"]      = url
        news["content"]  = soup_c.select("td.t_f > article")[0].text.replace('\n','').replace('發表','').strip()
        news["comments"] = []
        for i in range(len(soup_c.select("td.info-post-td > a"))):
            news["comments"].append({})   
            news["comments"][i]["comment_content"] = "".join([p.text for p in soup_c.select('[id^="postmessage_"] > div[align="left"]')])
            news["comments"][i]["datetime"] = soup_c.select("span.date-show-info")[0].text.replace('\n','').replace('發表','').strip()
            news["comments"][i]["user_id"] = soup_c.select("td.info-post-td > a")[0].text
        news_list.append(news)
        count += 1
        if 0 == count%2:  #這裏改成100，每100個post輸出一次
            output_as_json(count, 'wetalk', news_list)
            load_into_mongodb('testDatabase', 'testCollection', news_list)
            news_list[:] = []
            print('輸出一個檔案囉～')
            

In [13]:
def output_as_json(index, media, x):  
    """
    Output as json files
    """
    with open("./News_Crawler/{}/{}_{}.txt".format(media, media, index), 'w', encoding='UTF-8') as f:
        jd = json.dumps(x, ensure_ascii=False)
        f.write(jd)

In [14]:
def load_into_mongodb(db_name, collection_name, doc):
    db = client[db_name]
    collection = db[collection_name]
    collection.insert_many(doc)
    

In [15]:
def main():
    post_urls = []
    urls = get_url_links(post_urls, "1")
    save_posts_info(urls)
    
    

In [16]:
if __name__ == "__main__":        
    StopWatch.start()
    client = MongoClient('localhost', 27017, maxPoolSize=None) #這裡需要指名你要連線的MongoDB所在的ip和port
    main()
    StopWatch.split('')

輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
輸出一個檔案囉～
break point = time elapsed = 0:01:42.621248
